Using information available when a vacancy is opened, can we estimate how many days it will take to hire someone?

This project explores recruitment lifecycle data and prepares a clean target
variable (`time_to_hire_days`) for downstream regression modelling.

In [ ]:
# Preparing data so that the regression model can learn the relationship between early vacancy information (X) and time_to_hire_days (y)

import pandas as pd
import numpy as np
df = pd.read_excel("data.xlsx")
print(df.head())
print(df.columns)
print(df.shape)
print(df.info())

                                           DEPT_DESC DEPT_SHORT_DESC VACANCY  \
0  (7P) Office of the Assist. Regional Administra...               7  471551   
1  (7P) Office of the Assist. Regional Administra...               7  471811   
2  (7P) Office of the Assist. Regional Administra...               7  471811   
3  (7P) Office of the Assist. Regional Administra...               7  471811   
4  (7P) Office of the Assist. Regional Administra...               7  471811   

   APPLICATION_COUNT  HIRE_COUNT RECEIVED_DATE APPROVED_DATE  OPEN_DATE  \
0                 38           1    2004-08-18    2004-08-19 2004-08-19   
1                  9           1    2004-09-17    2004-09-20 2004-09-20   
2                  9           1    2004-09-17    2004-09-20 2004-09-20   
3                  9           1    2004-09-17    2004-09-20 2004-09-20   
4                  9           1    2004-09-17    2004-09-20 2004-09-20   

  CLOSE_DATE ISSUE_DATE REFERRAL_RETURNED POS_OFFERED    VAC_LOCATIO

In [ ]:
# Converting date columns to datetime

date_cols = [
    "OPEN_DATE",
    "HIRED_DATE",
    "CLOSE_DATE",
    "ISSUE_DATE",
    "APPROVED_DATE",
    "RECEIVED_DATE"
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")


In [ ]:
# Creating the target variable (y)

df["time_to_hire_days"] = (df["HIRED_DATE"] - df["OPEN_DATE"]).dt.days
print(df[["OPEN_DATE", "HIRED_DATE", "time_to_hire_days"]].head())



# Removing invalid data

df = df.dropna(subset=["time_to_hire_days"])
df = df[df["time_to_hire_days"] > 0]


   OPEN_DATE HIRED_DATE  time_to_hire_days
0 2004-08-19 2004-11-14                 87
1 2004-09-20 2004-11-14                 55
2 2004-09-20 2004-11-14                 55
3 2004-09-20 2004-11-14                 55
4 2004-09-20 2004-11-14                 55


In [ ]:
# Removing unused columns to avoid data leakage

leakage_cols = [
    "HIRED_DATE",
    "OPEN_DATE",
    "CLOSE_DATE",
    "ISSUE_DATE",
    "APPROVED_DATE",
    "RECEIVED_DATE"
]

df_model = df.drop(columns=leakage_cols)




In [ ]:
# Defining x & y

X = df_model.drop(columns=["time_to_hire_days"])
y = df_model["time_to_hire_days"]


# Dropping unnecessary columns
X = X.drop(columns=['REFERRAL_RETURNED', 'POS_OFFERED'], errors='ignore')

In [ ]:
# Identifying numeric vs categorical columns in X

numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

numeric_cols, categorical_cols


(['APPLICATION_COUNT', 'HIRE_COUNT'],
 ['DEPT_DESC', 'DEPT_SHORT_DESC', 'VACANCY', 'VAC_LOCATION', 'SERIES'])

In [ ]:
# Handling missing values

for col in numeric_cols:
    X[col] = X[col].fillna(X[col].median())
for col in categorical_cols:
    X[col] = X[col].fillna(X[col].mode()[0])



# Converting category labels into numeric columns

X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)



In [ ]:
# Building Random Forest regression model

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded,y,test_size=0.2,random_state=42)

from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=50,max_depth=10,random_state=42)
rf_model.fit(X_train, y_train)


RandomForestRegressor(max_depth=10, n_estimators=50, random_state=42)

In [ ]:
# Evaluating model with metrics

y_pred = rf_model.predict(X_test)
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae, r2


(23.307066295685292, 0.5365547104945836)

Interpretation

MAE ≈ 23 → predictions are ~23 days off on average

R² ≈ 0.54 → model explains ~54% of the variation

This is good performance for recruitment data.